In [1]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import LinearRegression

In [2]:
df_jan = pd.read_parquet(".\data\yellow_tripdata_2023-01.parquet")
df_feb = pd.read_parquet(".\data\yellow_tripdata_2023-02.parquet")

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\oluse\AppData\Local\Temp\ipykernel_8632\768222998.py:1: SyntaxWarning: invalid escape sequence '\d'
  df_jan = pd.read_parquet(".\data\yellow_tripdata_2023-01.parquet")
C:\Users\oluse\AppData\Local\Temp\ipykernel_8632\768222998.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_feb = pd.read_parquet(".\data\yellow_tripdata_2023-02.parquet")


In [3]:
df_jan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

For the January data, there are a total of 19 columns.

In [4]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [5]:
# df_feb.info()

In [6]:
df_jan['duration'] = df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime

#  get the duration of trips

In [7]:
df_jan.duration = df_jan.duration.apply(lambda td: td.total_seconds() / 60)

# converts the duration to minutes

In [8]:
# Calculate the standard deviation
std_dev = df_jan['duration'].std()

print("Standard Deviation of January 2023 Trips:", round(std_dev, 2), "minutes")

Standard Deviation of January 2023 Trips: 42.59 minutes


In [9]:
# Filter values between 1 and 60
filtered_data = df_jan['duration'][(df_jan['duration'] >= 1) & (df_jan['duration'] <= 60)]
# filtered_data

In [10]:
print(str(round(len(filtered_data)/len(df_jan['duration']) * 100, 2)) + "%")

98.12%


From the above, the filtered data, after removing outliers consist of approximately 92% of the original data.

Now, we apply One-hot encoding.

In [11]:
categorical = ['PULocationID', 'DOLocationID']
df_jan[categorical] = df_jan[categorical].astype(str)
train_dicts = df_jan[categorical].to_dict(orient='records')

In [12]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [13]:
X_train.shape

(3066766, 518)

The above is the dimension of the feature matrix.

In [14]:
target = 'duration'
y_train = df_jan[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

41.996497706831995

In [15]:

def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [16]:
df_val = read_dataframe('./data/yellow_tripdata_2023-02.parquet')

In [17]:
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [18]:
y_val = df_val[target].values

In [19]:
y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.945210075408408

The above is the validation error on application to the February data.